# Regridding Data



## Overview

CDMS provides several methods for interpolating gridded data:

-  from one rectangular, lat-lon grid to another (CDMS regridder)
-  between any two lat-lon grids (ESMF regridder)
-  from one set of pressure levels to another
-  from one vertical (lat/level) cross-section to another vertical
   cross-section.


CDMS Horizontal Regridder

The simplest method to regrid a variable from one rectangular, lat/lon
grid to another is to use the regrid function defined for variables.
This function takes the target grid as an argument, and returns the
variable regridded to the target grid:


In [ ]:
import os.path
if(not os.path.exists("clt.nc")):
    !wget http://cdat.llnl.gov/cdat/sample_data/clt.nc
    !wget http://cdat.llnl.gov/cdat/sample_data/sampleCurveGrid4.nc
    !wget http://cdat.llnl.gov/cdat/sample_data/geos5-sample.nc
    !wget http://cdat.llnl.gov/cdat/sample_data/xieArkin-T42.nc
    !wget http://cdat.llnl.gov/cdat/sample_data/tas_6h.nc
    !wget http://cdat.llnl.gov/cdat/sample_data/rmp_T42_to_POP43_conserv.nc
    !wget https://cdat.llnl.gov/cdat/sample_data/ta_ncep_87-6-88-4.nc
    !wget https://cdat.llnl.gov/cdat/sample_data/rmp_T42_to_C02562_conserv.nc

## First regridding example:

In [ ]:
import cdms2
f1=cdms2.open("clt.nc")
f2=cdms2.open("geos5-sample.nc")

clt=f1('clt')  # Read the data
print("input grid:",clt.shape)

ozone=f2['ozone']  # Get the file variable (no data read with square brackets)
outgrid = ozone.getGrid() # Get the target grid
print("desired grid:",outgrid.shape)
print("regridding input data...")

cltnew = clt.regrid(outgrid)
print("new regridded input data:",cltnew.shape)


## Regridder function

A somewhat more efficient method is to create a regridder function. This
has the advantage that the mapping is created ``only once`` and can be used for **multiple arrays**. Also, this method can be used with data in the form of an ``MV2.MaskedArray``. The steps in this process are:

* Given an input grid and output grid, generate a regridder function.
* Call the regridder function on a Numpy array, resulting in an array
   defined on the output grid. The regridder function can be called with
   any array or variable defined on the input grid.

### Efficient method using first example
The following example illustrates this process. 

1. Makes the Regridder class available from the regrid module.

In [ ]:
import cdms2
from regrid2 import Regridder

2. Opens the input dataset.
2. Gets the variable object named ‘clt’. No data is read using square brackets ``[]``.

In [ ]:
f = cdms2.open("clt.nc")
cltf = f['clt']
ingrid = cltf.getGrid() # Get the input grid

4. Opens a dataset to retrieve the output grid.

2. The output grid is the grid associated with the variable named **ozone** in dataset ``g``. ***Note***: Just the grid is retrieved, not the data using square brackets [].

In [ ]:
g = cdms2.open('geos5-sample.nc')
outgrid = g['ozone'].getGrid()  # Get the output grid

6. Generates a regridder function regridfunc.

In [ ]:
regridfunc = Regridder(ingrid, outgrid)  # Create the "Regridder function"

7. Calls the regridder
function on that data, all data for variable cltf will be read and execute the regridder function on that data, resulting in a transient variable cltnew.

In [ ]:
cltnew = regridfunc(cltf)
print(cltnew.shape)

8. Close files.

In [ ]:
f.close()
g.close()

# ESMF Horizontal Regridder

To interpolate between grids where one or both grids is non-rectangular,
CDMS provides an interface to the ESMF regridder package. (https://www.earthsystemcog.org/projects/esmf/). 


## Regridding Data



Create the regridder using remapper file.

In [ ]:
import regrid2, cdms2
# Read the regridder from the remapper file
remapf = cdms2.open('rmp_T42_to_POP43_conserv.nc')
regridf = regrid2.readRegridder(remapf)
remapf.close()

## Then read the input data and regrid

In [ ]:
# Get the source variable
f = cdms2.open('xieArkin-T42.nc')
t42prc = f('prc')
f.close()
# Regrid the source variable
popdat = regridf(t42prc)
print("input grid:", t42prc.shape)
print("output grid:",popdat.shape)

**Note** that ``t42dat`` can have rank greater than 2. The trailing
dimensions must match the input grid shape. 

For example, if ``t42dat``
has shape (216, 64, 128), then the input grid must have shape (64, 128).
Similarly if the variable had a generic grid with shape (128, 192) the
last dimension of the variable would have length (128, 192).

## Pressure-Level Regridder

To regrid a variable which is a function of latitude, longitude,
pressure level, and (optionally) time to a new set of pressure levels,
use the ``pressureRegrid`` function defined for variables. This function
takes an axis representing the target set of pressure levels, and
returns a new variable ``d`` regridded to that dimension.

In [ ]:
f=cdms2.open("ta_ncep_87-6-88-4.nc")
ta=f('ta')
print(ta.shape)
print(ta.getAxisIds())

result = ta.pressureRegrid(cdms2.createAxis([1000.0]))
print(result.shape)


## Cross-Section Regridder

To regrid a variable which is a function of latitude, height, and
(optionally) time to a new latitude/height cross-section, use the
``crossSectionRegridder`` defined for variables. This function takes as
arguments the new latitudes and heights, and returns the variable
regridded to those axes.

In [ ]:
f=cdms2.open("ta_ncep_87-6-88-4.nc")
ta=f('ta')
print(ta.shape)

levOut=cdms2.createAxis([1000.0,950.])
levOut.designateLevel()

latOut=cdms2.createAxis(ta.getLatitude()[10:20])
latOut.designateLatitude()

# Read frist time only
ta0 = ta[0,:]
print(ta0.getAxisIds())

# regrid to new latitude/height
taout = ta0.crossSectionRegrid(levOut, latOut)
print(taout.shape)

## Regrid Module

The ``regrid`` module implements the CDMS regridding functionality as
well as the ESMF interface. Although this module is not strictly a part
of CDMS, it is designed to work with CDMS objects.

### ESMF Regridder

ESMF regridder functions are created with the ``regrid.readRegridder``
function:

#### ESMF Regridder Constructor

 "``regridFunction = regrid.readRegridder(fileobj, mapMethod=None, checkGrid=1)``", "Read a regridder from an open CDMS file object.
* **fileobj** is a CDMS file object, as returned from **cdms.open**.
* **mapMethod** is one of:
 * **'conservative'**: conservative remapper, suitable where area-integrated fields such as water or heat fluxes must be conserved.
 * **'bilinear'**: bilinear interpolation
 * **'bicubic'**: bicubic interpolation
 * ***Note:*** It is only necessary to specify the map method if it is not defined in the file.
* If **checkGrid** is 1 (default), the grid cells are checked for convexity, and 'repaired' if necessary. Grid cells may appear to be nonconvex if they cross a ``0 / 2pi`` boundary. The repair consists of shifting the cell vertices to the same side modulo 360 degrees."


### ESMF Regridder Functions

An ESMF regridder function is an instance of the ScripRegridder class.
Such a function is created by calling the regrid.readRegridder method.
Typical usage is straightforward:

In [ ]:
import cdms2
import regrid2
remapf = cdms2.open('rmp_T42_to_POP43_conserv.nc')
regridf = regrid2.readRegridder(remapf)
f = cdms2.open('xieArkin-T42.nc')
t42prc = f('prc')
f.close()
# Regrid the source variable
popdat = regridf(t42prc)
print(t42prc.shape)
print(popdat.shape)

The bicubic regridder takes four arguments:

### Examples

Regrid data to a uniform output grid.

In [ ]:
import cdms2
from regrid2 import Regridder
f = cdms2.open('clt.nc')
cltf = f.variables['clt']
ingrid = cltf.getGrid()
outgrid = cdms2.createUniformGrid(90.0, 46, -4.0, 0.0, 72, 5.0)
regridFunc = Regridder(ingrid, outgrid)
newrls = regridFunc(cltf)
f.close()

### Regridder

* "3", "Open a netCDF file for input."
* "6", "Create a 4 x 5 degree output grid. Note that this grid is not associated with a file or dataset."
* "7", "Create the regridder function."
* "8", "Read all data and regrid. The missing data value is obtained from variable rlsf"

Return the area fraction of the source (input) grid cell that
participates in the regridding. The array is 1-D, with length equal to
the number of cells in the input grid.

Get a mask from a separate file, and set as the input grid mask.

In [ ]:
# wget http://cdat.llnl.gov/cdat/sample_data/clt.nc
# wget http://cdat.llnl.gov/cdat/sample_data/geos5-sample.nc
import cdms2
from regrid2 import Regridder
#
f = cdms2.open('clt.nc')
cltf = f.variables['clt']
outgrid = cltf.getGrid()

g = cdms2.open('geos5-sample.nc')
ozoneg = g.variables['ozone']
ingrid = ozoneg.getGrid()

regridFunc = Regridder(ingrid,outgrid)

uwmaskvar = g.variables['uwnd']
uwmask = uwmaskvar[:]<0
outArray = regridFunc(ozoneg.subSlice(time=0), mask=uwmask)
print(uwmask.shape)
print(outArray.mask.shape)
f.close()
g.close()

### Zonal mean regridder

In [ ]:
f = cdms2.open("clt.nc")
clt = f.variables["clt"]
ingrid = clt.getGrid()
outgrid = cdms2.createZonalGrid(ingrid)
regridFunc = Regridder(ingrid,outgrid)
mean = regridFunc(clt)
print(clt.shape)
print(mean.shape)
f.close()

In [ ]:
import cdms2
from cdms2.MV2 import *
from regrid2 import Regridder

f = cdms2.open("ta_ncep_87-6-88-4.nc")
var = f('ta')

outgrid = cdms2.createUniformGrid(90.0, 46, -4.0, 0.0, 72, 5.0)

outlatw, outlonw = outgrid.getWeights()
outweights = outerproduct(outlatw, outlonw)

grid = var.getGrid()

sample = var[0,0]

latw, lonw = grid.getWeights()
weights = outerproduct(latw, lonw)

inmask = where(greater(absolute(sample),1.e15),0,1)

mean = add.reduce(ravel(inmask*weights*sample))/add.reduce(ravel(inmask*weights))

regridFunc = Regridder(grid, outgrid)
outsample, outmask = regridFunc(sample, mask=inmask, returnTuple=1)
outmean = add.reduce(ravel(outmask*outweights*outsample)) / add.reduce(ravel(outmask*outweights))

Regrid from a curvilinear to a generic grid, using a conservative
remapping. Compute the area-weighted means on input and output for
comparison.

In [ ]:
import cdms2, regrid2, MV2

# Open the SCRIP remapping file and data file
fremap = cdms2.open('rmp_T42_to_C02562_conserv.nc')
fdat = cdms2.open('xieArkin-T42.nc')

# Input data array
dat = fdat('prc')[0,:]

# Read the ESMF regridder
regridf = regrid2.readRegridder(fremap)

# Regrid the variable
outdat = regridf(dat)

# Get the cell area and fraction arrays. Areas are computed only
# for conservative regridding.
srcfrac = regridf.getSourceFraction()
srcarea = regridf.getSourceArea()
dstfrac = regridf.getDestinationFraction()
dstarea = regridf.getDestinationArea()

# calculate area-weighted means
inmean = MV2.sum(srcfrac*srcarea*MV2.ravel(dat)) / MV2.sum(srcfrac*srcarea)
outmean = MV2.sum(dstfrac*dstarea*MV2.ravel(outdat)) / MV2.sum(dstfrac*dstarea)
print('Input mean:', inmean)
print('Output mean:', outmean)

fremap.close()
fdat.close()